In [52]:
testconfig={
	"base":"basefolder",
	"notebooks":[	
		{"name":"1 - Import UDF Simple Parser.ipynb",
			"tests":[{
					"locator":"0.0.text",
					"asserts":[
						{"eval":"x.split(' ')[3].isdigit()", "message":"generated file size should be integer"},
						{"eval":"int(x.split(' ')[3])>5000000", "message":"generated file size should be more than 5 mb"},
					]
				},{
					"locator":"2.0.text",
					"asserts":[
						{"eval":"len(x)>500", "message":"test records too short"},
						{"eval":"len(x.split('{')) == 3", "message":"expected 3 test records"},
					]
				},{
					"locator":"4.1.data.text/html",
					"asserts":[
						{"eval":"len(x)>10000", "message":"resultset html of import udf too short"},
						{"eval":"x.startswith('<table><tr><th>security</th>')", "message":"resultset html of import udf should start with 'security' column"},
					]
				},
			]
		},
		{"name":"2 - Import UDF Parser Debugging and Troubleshooting.ipynb",
        			"tests":[{
					"locator":"0.0.text",
					"asserts":[
						{"eval":"x.split(' ')[3].isdigit()", "message":"generated file size should be integer"},
						{"eval":"int(x.split(' ')[3])>3000000", "message":"generated file size should be more than 3 mb"},
					]
				},{
					"locator":"2.0.text",
					"asserts":[
						{"eval":"len(x)>500", "message":"test records too short"},
						{"eval":"len(x.split('{')) == 10", "message":"expected 10 sample records"},
						{"eval":"len(x.split('Bad Record')) > 10", "message":"expected some Bad Records"},
					]
				},{
					"locator":"5.1.data.text/html",
					"asserts":[
						{"eval":"len(x)>10000", "message":"resultset html of import udf test should be longer than 10 kb"},
						{"eval":"x.startswith('<table><tr><th>XXXXXXXsecurity</th>')", "message":"resultset html of import udf should start with 'security' column"},
					]
				},{
					"locator":"6.0.data.text/plain",
					"asserts":[
						{"eval":"len(x)>500", "message":"log records too short"},
						{"eval":"len(x.split('{')) >= 10", "message":"expected 10 logged records"},
						{"eval":"len(x.split('Source UDF')) > 0", "message":"no records logged from UDF"},
					]
				},
			]
		},
		{"name":"3 - Import UDF - Simple Connector.ipynb",
        	"tests":[{
					"locator":"0.0.text",
					"asserts":[
						{"eval":"x == 'Data has been generated!'", "message":"SQLite db not generated"},
					]
				},{
					"locator":"1.0.text",
					"asserts":[
						{"eval":"len(x)>500", "message":"test records too short"},
						{"eval":"len(x.split('(')) == 10", "message":"expected 10 sample records"},
					]
				},{
					"locator":"2.0.text",
					"asserts":[
						{"eval":"x.split(' ')[3].isdigit()", "message":"generated config file size should be integer"},
						{"eval":"int(x.split(' ')[3])>90", "message":"generated config file size should be more than 5 mb"},
					]
				},{
					"locator":"5.1.data.text/html",
					"asserts":[
						{"eval":"len(x)>10000", "message":"resultset html of import udf test should be longer than 10 kb"},
						{"eval":"x.startswith('<table><tr><th>security</th>')", "message":"resultset html of import udf should start with 'security' column"},
					]
				},
			]
        },
		{"name":"4 - Map UDF Example.ipynb",
            "tests":[{
					"locator":"2.0.text",
					"asserts":[
						{"eval":"len(x)>500", "message":"log records too short"},
						{"eval":"len(x.split('\\n')) >= 10", "message":"expected 10 logged records"},
					]
				},
			]},
		{"name":"5 - Map UDF Testing.ipynb",
            "tests":[{
					"locator":"4.0.text",
					"asserts":[
						{"eval":"len(x)>500", "message":"log records too short"},
						{"eval":"len(x.split('\\n')) >= 10", "message":"expected 10 logged records"},
					]
				},
			]},
		{"name":"6 - Map UDF Business Rules.ipynb",
            "tests":[{
					"locator":"2.0.text",
					"asserts":[
						{"eval":"len(x)>500", "message":"log records too short"},
						{"eval":"len(x.split('\\n')) >= 10", "message":"expected 10 logged records"},
					]
				},
			]}
	]
} 

#import ast #import literal_eval 

In [54]:
#cells = [cell['outputs'] for cell in nb['cells'] if cell['cell_type'] == 'code']
#test_corpus = testconfig["notebooks"][0]["tests"]
#test = test_corpus[0]

def get_location(locator, subject):
    try:
        operator=locator[0]
        if operator.isdigit():
            subject = subject[int(operator)]
        else:
            subject = subject[operator]
        if len(locator) > 1:
            return get_location(locator[1:],subject)
        else:
            return subject
    except Exception as e:
        print (str(e))
        print (locator)
        #print (subject)
        raise e
    
def checkNotebook(num, cells):
    for cell_check in testconfig["notebooks"][num]["tests"]:
        print (cell_check["locator"])
        x = get_location(cell_check["locator"].split("."), cells)
        for a in cell_check['asserts']:
            if (eval (a['eval']), a['message']) :
                print(a['message'], ' - ok')
            else:
                print('Error at', cell_check["locator"], ': ',a['message'])
                print('>>',x)
#subject    
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors import CellExecutionError

basepath = os.getcwd()
#ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
meta = {'metadata': {'path': basepath}}
ep = ExecutePreprocessor(timeout=600, kernel_name='python3')

#eee = None
nb = None

for i, notebook in enumerate(testconfig["notebooks"]):
    #if i != 1: continue
    print (i, os.path.join(basepath, notebook["name"])) 
    with open(os.path.join(basepath, notebook["name"])) as f:
        nb = nbformat.read(f, as_version=4)
        try:
            ep.allow_errors=True
            out = ep.preprocess(nb, meta)
            cells = [cell['outputs'] for cell in nb['cells'] if cell['cell_type'] == 'code']
            errors = [(i, [(o["ename"],o["evalue"]) for o in c if o.get("output_type")=="error"]) for (i, c) in enumerate(cells)]
            errors = [{"cell":x[0],"errors":x[1]} for x in errors if len(x[1]) > 0]
            if len(errors) > 0:
                print(errors)
            else:
                print('>>> No Cell Errors!')

            checkNotebook(i, cells)
            
            print ('Analzyed',len(out[0]['cells']),'cells\n')
            
        except CellExecutionError as e:
            out = None
            msg = 'Error!!! executing the notebook "%s".\n\n' % str(e)
#            eee = e
            print(msg)
        except Exception as e:
            out = None
            msg = 'Error??? "%s".\n\n' % str(e)
            print(msg)
            raise(e)
            
#             raise
#         finally:
#             with open(notebook_filename_out, mode='wt') as f:
#                 nbformat.write(nb, f)
        


0 /var/opt/xcalar/jupyterNotebooks/xdpadmin-TutorialNotebooks-HelloUDF-Full/1 - Import UDF Simple Parser.ipynb

0.0.text
'text'
['text']
'text'
['0', 'text']
'text'
['0', '0', 'text']
Error??? "'text'".




KeyError: 'text'

In [44]:
cells = [cell['outputs'] for cell in nb['cells'] if cell['cell_type'] == 'code']


In [51]:
def getConfigDict():
    from xcalar.compute.api.Env import XcalarConfigPath
    localExportDir = XcalarConfigPath
    cfgData = None
    with open(XcalarConfigPath, 'r') as f:
        cfgData = f.read()
    configdict = {}
    for line in cfgData.splitlines():
        if "=" in line:
            name, var = line.partition("=")[::2]
            configdict[name] = var.strip()
    return configdict
config = getConfigDict()
jnpath = os.getcwd()
logpath = os.path.join(jnpath.split("opt/xcalar/jupyterNotebooks")[0],"log/xcalar/xpu.out")
path = config.get('Constants.XcalarLogCompletePath',logpath)
path

'/var/log/xcalar/xpu.out'